In [1]:
# 引入依赖
import cracknuts as cn

# 创建 cracker 
cracker = cn.new_cracker('192.168.0.15')

In [2]:
cracker.connect(force_update_bin=True, force_write_default_config=True)

In [3]:
cracker.get_id() # 获取设备ID

(0, '75342125079585')

In [4]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(c):
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracer, count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


def finish(c):
    ...
    # print('optional behavior')


acq = cn.new_acquisition(cracker, init=init, do=do, finish=finish)

In [5]:
cn.panel(acq)

CracknutsPanelWidget(acq_run_progress={'finished': 0, 'total': -1}, connect_status=True, custom_y_range={'0': …

[WARNING] 2025-11-12 12:53:04,851 cracker_basic.send_and_receive:525 Command 0x0263 with payload 00 06 00 00 00 03 e8 01 00 00 00 00 00 00 10 11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff received a non-OK response with status code 0x0002, and payload        .


In [ ]:
cracker.set_logging_level('debug')